## Scrapper de Leilões de carros
### Coisas importantes:
1. Modelo do carro
2. Ano do carro
3. Sinistro
4. Quilometragem
5. Data do leilão
6. Link para o carro

### Segunda etapa
1. Pelo link pegar o preco do maior lance
2. incremento min

Entra no link, pega todas informações de cada carro
passa pra proxima página, até a ultima
se achar algum que bate com o nome colocado de entrada
ele separa as informações e envia via WhatsApp

In [42]:
#-- IMPORT LIBS
from re import search
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
from pywhatkit import sendwhatmsg
import csv


def get_time() -> tuple:
    """
    Funcao para pegar a hora
    do local em que voce esta
    :return:
    """
    currentDateAndTime = datetime.now()

    currentTime_hours = currentDateAndTime.strftime("%H")
    currentTime_minutes = currentDateAndTime.strftime("%M")

    return currentTime_hours, currentTime_minutes

def navegador(start_url:str) -> any:
    """
    Funcao para iniciar o navegador chrome
    :param start_url:
    :return:
    """
    #-- OPTIONS TO START BROWSER
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--ignore-certificate-errors')

    #-- START BROWSER
    print('Abrindo o navegador...')
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.get(start_url)

    return driver

def wpp(celular:str, msg:str) -> None:
    """
    Funcao para enviar msg no wpp,
    Quando for colocar o numero precisa do codigo
    do pais +55
    :param celular, msg:
    :return:
    """
    print('Em alguns segundos WhatsApp vai abrir e em 15 segundos enviar sua msg!')
    hora, minuto = get_time()
    sendwhatmsg(celular, msg, int(hora), int(minuto)+2, tab_close=True)

    return

def create_log() -> None:
    """
    Funcao para criar arquivo de logs
    :return:
    """
    with open('/logs.csv', 'w') as f:
        writer = csv.DictWriter(f, fieldnames = ["modelo", "versao", "data_leilao",\
                                                      "hora_leilao", "preco_inicial", "ano_carro",\
                                                      "lance_atual", "link"])
        writer.writeheader()
        f.close()

    return


def log(nome_carro:str,
        modelo:str,
        data_leilao:str,
        hora_leilao:str,
        preco_inicial:str,
        ano_carro:str,
        lance_atual:str,
        link:str) -> None:
    """
    Funcao para adicionar arquivo de logs,
    para nao repetir carros ja enviados
    :param nome_carro, modelo, data_leilao, hora_leilao, preco_inicial, ano_carro, lance_atual, link:
    :return:
    """
    with open('/logs.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow([nome_carro, modelo, data_leilao, hora_leilao, preco_inicial, ano_carro, lance_atual, link])
        f.close()

    return


def scrapper(modelo_carro:str):
    """
    Scrapper para comecar a verificar os sites,
    Use o modelo do carro em letras maiusculas
    :param modelo_carro:
    :return:
    """
    create_log()
    for pag in range(20):

        driver = navegador('https://www.freitasleiloeiro.com.br/leiloes/pesquisar?pg=1&categoria=1')

        # Procura as descricoes dos carros
        carros = driver.find_elements(By.CLASS_NAME, 'cursor-pointer ')

        # Patterns para o regex search
        nome_carro = r'ONIX'
        pattern_model = r'(?=ONIX).+?(?<=,)'
        pattern_link_model = r"(?=').+(?<=')"
        pattern_data = r"[0-9]{2}\/[0-9]{2}\/[0-9]{4}"
        pattern_hora_leilao = r'[0-9]{2}:[0-9]{2}'
        patterno_preco_inicial = r'(?<=R\$ ).+'
        pattern_ano_carro = r"(?<=,).+?(?<=,)"
        aspas = r"'"
        vazio = ""


        for i in carros:
            if search(nome_carro, i.text):
                # Separa as info do carro que foi encontrado
                link = search(pattern_link_model, i.get_attribute("onclick")).group(0)
                link_carro = f"https://www.freitasleiloeiro.com.br{link.replace(aspas, vazio)}"
                modelo = search(pattern_model, i.text).group(0)
                data_leilao = search(pattern_data, i.text).group(0)
                hora_leilao = search(pattern_hora_leilao, i.text).group(0)
                preco_inicial = search(patterno_preco_inicial, i.text).group(0)
                ano_carro = search(pattern_ano_carro, i.text).group(0)

                # Entra no link do carro que achou e pega o lance atual
                driver.get(link_carro)
                lance_atual = driver.find_element(By.ID, 'lance-atual').text
                driver.close()

                # Insere o log do carro que achou
                log(nome_carro, modelo, data_leilao, hora_leilao, preco_inicial, ano_carro,lance_atual, link)

                # Envia mensagem no Wpp com os dados do carro e link
                mensagem = f"Oportunidade!\n{modelo} {ano_carro}\nPreco inicial: {preco_inicial}\n{lance_atual}\nLeilao no dia: {data_leilao} as {hora_leilao}\nConfira o carro no link: {link_carro}"

                wpp("+5514996601344", mensagem)

    return

In [40]:

driver = navegador('https://www.freitasleiloeiro.com.br/leiloes/pesquisar?pg=1&categoria=1')

carros = driver.find_elements(By.CLASS_NAME, 'cursor-pointer ')

nome_carro = r'ONIX'
pattern_model = r'(?=ONIX).+?(?<=,)'
pattern_link_model = r"(?=').+(?<=')"
pattern_data = r"[0-9]{2}\/[0-9]{2}\/[0-9]{4}"
pattern_hora_leilao = r'[0-9]{2}:[0-9]{2}'
patterno_preco_inicial = r'(?<=R\$ ).+'
pattern_ano_carro = r"(?<=,).+?(?<=,)"
aspas = r"'"
vazio = ""


for i in carros:
    if search(nome_carro, i.text):
        link = search(pattern_link_model, i.get_attribute("onclick")).group(0)
        link_carro = f"https://www.freitasleiloeiro.com.br{link.replace(aspas, vazio)}"
        modelo = search(pattern_model, i.text).group(0)
        data_leilao = search(pattern_data, i.text).group(0)
        hora_leilao = search(pattern_hora_leilao, i.text).group(0)
        preco_inicial = search(patterno_preco_inicial, i.text).group(0)
        ano_carro = search(pattern_ano_carro, i.text).group(0)

        driver.get(link_carro)
        lance_atual = driver.find_element(By.ID, 'lance-atual').text
        driver.close()

        # with open('/', 'a')

        mensagem = f"Oportunidade!\n{modelo} {ano_carro}\nPreco inicial: {preco_inicial}\n{lance_atual}\nLeilao no dia: {data_leilao} as {hora_leilao}\nConfira o carro no link: {link_carro}"

        wpp("+55", mensagem)

        break

Abrindo o navegador...


C:\Users\flow igoulart\AppData\Local\Temp\ipykernel_9268\3871398392.py:36: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


Em alguns segundos WhatsApp vai abrir e em 15 segundos enviar sua msg!
In 104 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!


'7'